In [40]:
# All imports go here
import polars as pl
from time import time

In [41]:
import adbc_driver_postgresql.dbapi
from adbc_driver_manager.dbapi import Connection

In [42]:
# Constants
user="postgres"
password="postgres"
db_name = "ny_taxi"
port=5433
hostname="localhost"
DATABASE_URL = f"postgresql://{user}:{password}@{hostname}:{port}/{db_name}"
# TABLE_NAME="yellow_taxi_data_polars"
TABLE_NAME="green_taxi_data_polars"

# ENDPOINT = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-01.csv.gz"
# ENDPOINT = "hw1/data/yellow_tripdata_2019-01.csv.gz"
# ENDPOINT = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
ENDPOINT = "hw1/data/green_tripdata_2019-10.csv.gz"


In [43]:
# Create a cursor instance
conn: Connection = adbc_driver_postgresql.dbapi.connect(DATABASE_URL)

In [44]:
def yellow_taxi_polars_schema() -> dict:
  return {
    "VendorID": pl.Int32,
    "tpep_pickup_datetime": pl.Datetime,
    "tpep_dropoff_datetime": pl.Datetime,
    "passenger_count": pl.Int8,
    "trip_distance": pl.Float64,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "RatecodeID": pl.Int8,
    "store_and_fwd_flag": pl.String,
    "payment_type": pl.Int8,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "total_amount": pl.Float64,
    "congestion_surcharge": pl.Float64,
  }

In [45]:
def green_taxi_polars_schema() -> dict:
  return {
    "VendorID": pl.Int32,
    "lpep_pickup_datetime": pl.Datetime,
    "lpep_dropoff_datetime": pl.Datetime,
    "store_and_fwd_flag": pl.String,
    "RatecodeID": pl.Int8,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "passenger_count": pl.Int8,
    "trip_distance": pl.Float64,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "ehail_fee": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "total_amount": pl.Float64,
    "payment_type": pl.Int8,
    "trip_type" : pl.Int8,
    "congestion_surcharge": pl.Float64,
  }

In [46]:
# df = pl.read_csv(ENDPOINT, dtypes=green_taxi_polars_schema(), n_rows=100)
# df = pl.read_csv(ENDPOINT, schema_overrides=green_taxi_polars_schema(), n_rows=1000)

In [47]:
# df

In [48]:
def recreate_table(conn: Connection, schema: dict, table_name: str) -> None:
    """
    Creates a PostgreSQL table for yellow taxi data using ADBC engine.
    
    Args:
        connection_string: PostgreSQL connection string
        table_name: Name of the table to create
    """
    # Map Polars types to PostgreSQL types
    pg_type_mapping = {
        pl.Int32: "INTEGER",
        pl.Int8: "SMALLINT",
        pl.Float64: "DOUBLE PRECISION",
        pl.Datetime: "TIMESTAMP",
        pl.String: "VARCHAR"
    }

    
    # Convert schema to PostgreSQL column definitions
    columns = []
    for column_name, polars_type in schema.items():
        quoted_column_name = f'"{column_name}"'
        pg_type = pg_type_mapping[polars_type]
        columns.append(f"{quoted_column_name} {pg_type}")
    
    # Drop the table if it already exists
    drop_table_sql = f"DROP TABLE IF EXISTS {table_name};"
    print(f"Drop table sql ==> {drop_table_sql}")
    # Create a cursor instance
    with conn.cursor() as cur:
        cur.execute(drop_table_sql)
        conn.commit()
    
    # Create the table definition SQL
    nl = ",\n        "
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {nl.join(columns)}
        );
        """
    
    print(f"Create table sql ==> {create_table_sql}")
    
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()

In [57]:
recreate_table(conn, green_taxi_polars_schema(), "green_taxi_data_polars")

Drop table sql ==> DROP TABLE IF EXISTS green_taxi_data_polars;
Create table sql ==> 
        CREATE TABLE IF NOT EXISTS green_taxi_data_polars (
            "VendorID" INTEGER,
        "lpep_pickup_datetime" TIMESTAMP,
        "lpep_dropoff_datetime" TIMESTAMP,
        "store_and_fwd_flag" VARCHAR,
        "RatecodeID" SMALLINT,
        "PULocationID" INTEGER,
        "DOLocationID" INTEGER,
        "passenger_count" SMALLINT,
        "trip_distance" DOUBLE PRECISION,
        "fare_amount" DOUBLE PRECISION,
        "extra" DOUBLE PRECISION,
        "mta_tax" DOUBLE PRECISION,
        "tip_amount" DOUBLE PRECISION,
        "tolls_amount" DOUBLE PRECISION,
        "ehail_fee" DOUBLE PRECISION,
        "improvement_surcharge" DOUBLE PRECISION,
        "total_amount" DOUBLE PRECISION,
        "payment_type" SMALLINT,
        "trip_type" SMALLINT,
        "congestion_surcharge" DOUBLE PRECISION
        );
        


In [54]:
def batch_read_and_insert_into_db(schema: dict, table_name: str) -> None:
  batch_reader = pl.read_csv_batched(
      source = ENDPOINT,
      batch_size=100000,
      schema_overrides=schema,
      has_header=True,
      separator=",",
      try_parse_dates=True
  )

  batches = batch_reader.next_batches(n=10000)

  start_time = time()
  for df in batches:
    t_start = time()
    df.write_database(
      table_name,
      connection=conn,
      engine="adbc",
      if_table_exists="append",
    )
    t_end = time()
    print('inserted another chunk, took %.3f second' % (t_end - t_start))

  end_time = time()
  print(f"Final insertion took {end_time - start_time} seconds")

In [51]:
def get_row_count(table_name: str)->int:
  with conn.cursor() as cur:
    cur.execute(f"SELECT count(1) FROM {table_name};")
    # print(cur.fetch_arrow_table())
    return cur.fetchone()[0]

In [61]:
batch_read_and_insert_into_db(green_taxi_polars_schema(), TABLE_NAME)

inserted another chunk, took 0.027 second
inserted another chunk, took 0.012 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.014 second
inserted another chunk, took 0.012 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.012 second
inserted another chunk, took 0.012 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.010 second
inserted another chunk, took 0.010 second
inserted another chunk, took 0.010 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.009 second
inserted another chunk, took 0.010 second
inserted another chunk, took 0.010 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.011 second
inserted another chunk, took 0.012

In [62]:
get_row_count(TABLE_NAME)

476386